### ***Installing the required dependencies***

In [9]:
!pip install transformers datasets evaluate sacrebleu

### ***Importing required modules for model training and evaluation***

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np

### ***Loading the dataset for English to Spanish translation task***

In [11]:
dataset = load_dataset("opus_books", "en-es")  # English to Spanish translation dataset

### ***Loading pre-trained model and tokenizer from Hugging Face Model Hub***

In [12]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es")

### ***Preprocessing function to prepare the dataset for the model***

In [13]:
def preprocess_function(examples):
    inputs = [item["en"] for item in examples["translation"]]
    targets = [item["es"] for item in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

### ***Splitting the tokenized dataset into training and test sets***

In [15]:
tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)

### ***Importing additional tools for and trainer configuration***

In [ ]:
from transformers import EarlyStoppingCallback, Trainer, TrainingArguments

# Defining training arguments for the Trainer class
training_args = TrainingArguments(
    output_dir='./results',              # Output directory
    eval_strategy="epoch",               # Evaluate every epoch
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=16,      # Training batch size
    per_device_eval_batch_size=16,       # Eval batch size
    num_train_epochs=3,                  # Recommended epoch
    weight_decay=0.01,                   # Weight decay
)

# Define Trainer with EarlyStopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

# Start Training
trainer.train()

<ipython-input-18-be5bec22cefc>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.683900,0.621844
2,0.620800,0.577980
3,0.577300,0.567580


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=14022, training_loss=0.6689967764986512, metrics={'train_runtime': 6349.7459, 'train_samples_per_second': 35.329, 'train_steps_per_second': 2.208, 'total_flos': 7604352407568384.0, 'train_loss': 0.6689967764986512, 'epoch': 3.0})

### ***Saving the fine-tuned model and tokenizer***

In [19]:
model.save_pretrained('fine-tuned-model')
tokenizer.save_pretrained('fine-tuned-tokenizer')

('fine-tuned-tokenizer/tokenizer_config.json',
 'fine-tuned-tokenizer/special_tokens_map.json',
 'fine-tuned-tokenizer/vocab.json',
 'fine-tuned-tokenizer/source.spm',
 'fine-tuned-tokenizer/target.spm',
 'fine-tuned-tokenizer/added_tokens.json')

### ***Loading the fine-tuned model and tokenizer for inference***

In [20]:
# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained('./fine-tuned-tokenizer')
model = AutoModelForSeq2SeqLM.from_pretrained('./fine-tuned-model')

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


### ***Example input text in English***

In [ ]:
input_text = "Hello, how are you today?"

In [22]:
# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

In [23]:
# Generate the translation (English to Spanish)
translated_tokens = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)

In [24]:
# Decode the translated tokens back to text (Spanish)
translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

In [29]:
translated_text

'¿Cómo estás hoy?'


### ***Importing additional libraries for file handling in Google Colab***

In [25]:
from google.colab import files
import shutil

In [26]:
# Create a zip file of the saved model
shutil.make_archive('/content/fine-tuned-model', 'zip', '/content', 'fine-tuned-model')

# Download the zip file
files.download('/content/fine-tuned-model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
# Create a zip file of the saved tokenizer
shutil.make_archive('/content/fine-tuned-tokenizer', 'zip', '/content', 'fine-tuned-tokenizer')

# Download the zip file
files.download('/content/fine-tuned-tokenizer.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
import torch
torch.cuda.empty_cache()  # Clears unused GPU memory